### Determine Optimal Portfolio Allocation Using Monte Carlo

In [44]:
import datetime as dt
import yfinance as yf
import numpy as np
import pandas as pd

# 10 Companies to Design Portfolio Around
tickers = ["coke", 'googl', 'hd', 'ford', 'smg', 'jnj', 'ag']
start = "2000-01-01"
end = "2020-01-01"

# Grab yfinance Data
df_portfolio = yf.download(tickers, start, end).reset_index()

[*********************100%***********************]  7 of 7 completed


In [80]:
# Set Date as Index
df_stock_data = df_portfolio.set_index("Date").copy(deep=True)
df_stock_data = df_stock_data["Adj Close"].dropna()

companies = df_stock_data.columns
company_returns = []

for company in companies:
    df_stock_data[f"{company}".lower()] = np.log(df_stock_data[company] / df_stock_data[company].shift(1))
    company_returns.append(f"{company}".lower())

df_stock_data.dropna(inplace=True)

df_returns = df_stock_data[company_returns].copy(deep=True)
df_returns

,ag,coke,ford,googl,hd,jnj,smg
Date,,,,,,,
2006-11-29,0.005115,-0.003918,0.006486,-0.009958,0.014728,0.003481,0.001213
2006-11-30,0.005089,-0.008674,-0.019587,0.000330,0.009261,-0.004391,-0.000606
2006-12-01,-0.033552,-0.021617,-0.035798,-0.008306,0.025996,0.000911,0.000202
2006-12-04,0.020780,0.027460,0.013575,0.008388,0.005884,0.004688,0.003832
2006-12-05,-0.031334,0.011274,-0.018141,0.004425,0.004327,-0.001661,0.001610
...,...,...,...,...,...,...,...
2019-12-24,0.062979,0.005742,-0.020001,-0.004601,0.006638,-0.003489,0.000844
2019-12-26,0.016584,0.021414,0.039609,0.013329,0.000680,-0.000685,-0.000188
2019-12-27,-0.022454,0.003445,-0.039609,-0.005763,-0.003857,-0.000549,0.001500


### Calculate Mean Returns

In [81]:
# Calculating Mean Returns
ag_mean = df_returns["ag"].mean() * 252
coke_mean = df_returns["coke"].mean() * 252
ford_mean = df_returns["ford"].mean() * 252
googl_mean = df_returns["googl"].mean() * 252
hd_mean = df_returns["hd"].mean() * 252
jnj_mean = df_returns["jnj"].mean() * 252
smg_mean = df_returns["smg"].mean() * 252

return_means = [ag_mean, coke_mean, ford_mean, googl_mean, hd_mean, jnj_mean, smg_mean]

i = 0
new_df = {}
new_df_list = []
for ticker in tickers:
    new_df[ticker] = return_means[i]
    i += 1
new_df_list.append(new_df)

df_returns_mean = pd.DataFrame(new_df_list)
df_returns_mean

,coke,googl,hd,ford,smg,jnj,ag
0,0.087624,0.127026,-0.117682,0.129958,0.161476,0.090385,0.095211


In [89]:
# Calculating Covariance Matrix
df_covariance_matrix = df_returns.cov() * 252
df_covariance_matrix

,ag,coke,ford,googl,hd,jnj,smg
ag,0.437106,0.016288,0.027124,0.023022,0.017172,0.011987,0.020391
coke,0.016288,0.105159,0.013460,0.023077,0.027029,0.016193,0.023157
ford,0.027124,0.013460,0.564789,0.022540,0.015181,0.005729,0.014102
googl,0.023022,0.023077,0.022540,0.079600,0.033294,0.018724,0.027946
hd,0.017172,0.027029,0.015181,0.033294,0.065560,0.019263,0.034369
jnj,0.011987,0.016193,0.005729,0.018724,0.019263,0.027399,0.016132
smg,0.020391,0.023157,0.014102,0.027946,0.034369,0.016132,0.096138


In [90]:
i = 0
new_df = {}
new_df_list = []
for ticker in tickers:
    new_df[ticker] = 1/len(tickers)
    i += 1
new_df_list.append(new_df)

allocation = pd.DataFrame(new_df_list)
allocation

,coke,googl,hd,ford,smg,jnj,ag
0,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857


In [91]:
df_returns_mean

,coke,googl,hd,ford,smg,jnj,ag
0,0.087624,0.127026,-0.117682,0.129958,0.161476,0.090385,0.095211


In [92]:
expected_return = (df_returns_mean * allocation).sum().sum()
expected_return

0.0819994971034287

In [93]:
from functools import reduce
portfolio_risk = np.sqrt(reduce(np.dot, [allocation, df_covariance_matrix, allocation.T]))
portfolio_risk

array([[0.21324114]])

In [94]:
risk_free_rate = .035

# Sharpe Ratio
(expected_return - risk_free_rate) / portfolio_risk

array([[0.22040539]])